Most heterogeneous agent models can be expressed in terms of three functions

$$
\begin{align}
v_t =& v(\tilde{x}, v_{t+1})\\
d_t =& d(\tilde{x}, \tilde{v}, d_{t-1})\\
0 =& f(\tilde{x}, \tilde{d}, \tilde{v})
\end{align}
$$

where $x_t$ is the time-$t$ vector of _aggregate_ variables, $v_t$ is the agents decision functions over the complete state space and $d_t$ is the distribution accross agents.
Let, for the sake of generality, letters with a tolde denote the complete expected time series of a variable, e.g $\tilde{x} = \{x_{t}\}_{t=0}^\infty$.

